In [1]:
import pandas as pd
import altair as alt

In [2]:
dados_federais_campina = pd.read_csv("data\\dados_casos_obitos_cg.csv", encoding='UTF-8', error_bad_lines=False)
dados_municipais_campina = pd.read_csv("data\\dados_municipais_cg.csv", encoding='utf-8', index_col='data')

C:\Users\perei\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


## Gerando dados sobre Casos Confirmados por Dia (Dados Federais)

In [3]:
# Criando um data com a coluna 'data' e a coluna 'casosNovos'
df_casosPorDia = dados_federais_campina[['data', 'casosNovos']]

In [4]:
# Criando um DataFrame com os dados para o altair
df_casosPorDia = pd.DataFrame({
    'data': df_casosPorDia['data'].tolist(),
    'casos_confirmados': df_casosPorDia['casosNovos'].tolist()
})
# Criando uma nova coluna no DataFrame referente a média movel.
df_casosPorDia['media_movel'] = df_casosPorDia.casos_confirmados.rolling(7).mean()

In [5]:
# Gerando o Gráfico de Obitos por Dia com os dados Federais
graph_df_casosPorDia = alt.Chart(df_casosPorDia).mark_line().encode(
    alt.X('data:T', title='Data'),
    alt.Y('casos_confirmados:Q', title='Casos Confirmados'),
    color=alt.value('dodgerblue')
).properties(
    width=800,
    title='Casos Confirmados por Dia (Base Federal)'
)

graph_df_casosPorDia

alt.Chart(...)

In [6]:
# Gerando o grafico de média movel
graph_df_casosMedMovel = alt.Chart(df_casosPorDia).mark_line(
    point=True
).encode(
    alt.X('data:T', title='Data'),
    alt.Y('media_movel:Q', title='Casos Confirmados'),
    color=alt.value('orange')
)

In [7]:
graph_df_cmm = graph_df_casosPorDia + graph_df_casosMedMovel
graph_df_cmm

alt.LayerChart(...)

## Gerando dados sobre Casos Confirmados por Dia (Dados Municipal)

In [8]:
# Criando um data com a coluna 'data' e a coluna 'confirmadosPorDia'
dm_casosPorDia = dados_municipais_campina['confirmadosPorDia']

In [9]:
# Criando um DataFrame com os dados para o altair
dm_casosPorDia = pd.DataFrame({
    'data': dm_casosPorDia.index.tolist(),
    'casos_confirmados': dm_casosPorDia.values.tolist()
})
# Criando uma nova coluna no DataFrame referente a média movel.
dm_casosPorDia['media_movel'] = dm_casosPorDia.casos_confirmados.rolling(7).mean()

In [10]:
graph_dm_casosPorDia = alt.Chart(dm_casosPorDia).mark_line().encode(
    alt.X('data:T', title='Data'),
    alt.Y('casos_confirmados:Q', title='Casos Confirmados'),
    color=alt.value('darkorange')
).properties(
    width=800,
    title='Casos Confirmados por Dia (Base Municipal)'
)

graph_dm_casosPorDia

alt.Chart(...)

In [11]:
# Gerando o grafico de média movel
graph_dm_casosMedMovel = alt.Chart(dm_casosPorDia).mark_line(
    point=True
).encode(
    alt.X('data:T', title='Data'),
    alt.Y('media_movel:Q', title='Casos Confirmados'),
    color=alt.value('black')
)

In [12]:
graph_dm_cmm = graph_dm_casosPorDia + graph_dm_casosMedMovel
graph_dm_cmm

alt.LayerChart(...)

# Comparando os dados Federais e Municipais

In [13]:
graph_comp_casosPorDia = graph_df_casosPorDia + graph_dm_casosPorDia
graph_comp_casosPorDia.properties(
    title='Casos Confirmados Por Dia (Comparação)'
)

alt.LayerChart(...)

In [14]:
graph_comp_mediamovel = graph_df_casosMedMovel + graph_dm_casosMedMovel
graph_comp_mediamovel.properties(
    width=800,
    title='Média Movel de Casos Confirmados por Dia (Comparação)'
)

alt.LayerChart(...)

In [15]:
df_casosTotais = dados_federais_campina['casosAcumulado'].iloc[-1]
dm_casosTotais = dados_municipais_campina['confirmadosAcumulados'].iloc[-1]

In [16]:
totalCasos = pd.DataFrame({
    'base': ['Federal', 'Municipal'],
    'total': [df_casosTotais, dm_casosTotais]
})

In [17]:
alt.Chart(totalCasos).mark_bar(size=30).encode(
    alt.X('base:N', title='Base dos Dados'),
    alt.Y('total:Q', title='Nº Total de Casos Confirmados'),
    alt.Color('base:N', title='Instância')
).properties(
    width=150,
    title='Total de Casos Confirmados (Comparação)'
)

alt.Chart(...)